In [4]:
import numpy as np
import os

In [5]:
def read_xyz(xyz):
    coordlist=[]
    with open(xyz,'r') as inp:
        i=0
        for line in inp:   
            if i>=2:
                splitline=line.split()
                coordlist.append(splitline[1:4])
            i+=1
    return np.array(coordlist).astype(float)

def write_xyz(coordlist,name):
    with open(name,'w') as f:
        f.write(str(len(coordlist))+'\n#\n')
        for cd in coordlist:
            f.write('C '+str(cd[0])+' '+str(cd[1])+' '+str(cd[2])+'\n')
    
def write_xyz_stack(path,filenamearray):
    with open(path+'xyz_stack.xyz','w') as f:
        for file in filenamearray:
            cl=read_xyz(file)
            f.write(str(len(cl))+'\n'+filename+'\n')
            for coords in cl:
                f.write('C '+str(coords[0])+' '+str(coords[1])+' '+str(coords[2])+'\n')
        

In [6]:
## read xyz files ##
path='tests_small/'
file2=path+'relaxed_new.xyz'
filename='master.xyz'

filenamearray=[]
##loop over all master.xyz
#file1='master.xyz'
k=0
for subdir, dirs, files in sorted(os.walk(path)):
    for file in files:
        if file==filename:
            f=os.path.join(subdir, file)
            print('reading '+f)
            filenamearray.append(f)
            file1=f
            cl1=read_xyz(file1)
            cl2=read_xyz(file2)  

            ## get some atoms with same id (manually) ##
            same_ids=np.array([[1,8,249,229,242,17],[506,504,897,866,851,467]])
            same_ids-=1


            ## Rotate struture if necessairy ##
            xrotate=False
            if xrotate:
                ys=cl2[:,1]*np.cos(3.1415)-cl2[:,2]*np.sin(3.1415)
                zs=cl2[:,1]*np.sin(3.1415)+cl2[:,2]*np.cos(3.1415)

                cl2[:,1]=ys
                cl2[:,2]=zs

            ## calculate translation vector from correlated atoms ##    
            vec=np.zeros(3)
            for i in range(len(same_ids[0])):
                vec+=(cl1[same_ids[0,i]]-cl2[same_ids[1,i]])
            vec/=i+1

            ## apply translation ##
            cl2+=vec



            ## correlate all atoms ##
            ids=[]
            for coords in cl1:
                diff=np.sqrt(np.sum((coords-cl2)**2,axis=1))
                d=diff[np.argmin(diff)]
                ids.append(np.argmin(diff))


            ## calculate and apply translation of all correlated atoms ##
            ## this finetunes the translation ##
            vec=np.zeros(3)
            for i in range(len(cl1)):
                vec+=(cl1[i]-cl2[ids[i]])
            vec/=i+1
            cl2+=vec

            ## calculate difference ##

            totdiff=0
            for i in range(len(cl1)):
                totdiff+=np.sqrt(np.sum((cl1[i]-cl2[ids[i]])**2))
            print(totdiff/len(cl1))

            ## write new xyz ##
            #write_xyz(cl2,path+'relaxed_shifted'+str(k)+'.xyz')
            #k+=1
    
write_xyz_stack(path,filenamearray)

reading tests_small/master.xyz
0.22170248600671402
reading tests_small/010/master.xyz
0.18913477996312433
reading tests_small/020/master.xyz
0.17185053292072405
reading tests_small/030/master.xyz
0.16049410466327063
reading tests_small/040/master.xyz
0.15158427270165006
reading tests_small/050/master.xyz
0.1437478201490761
reading tests_small/060/master.xyz
0.1364034125176374
reading tests_small/070/master.xyz
0.1293802358542264
reading tests_small/080/master.xyz
0.1230021850448824
reading tests_small/090/master.xyz
0.12013504879611761
reading tests_small/095/master.xyz
0.1284110090498279
reading tests_small/100/master.xyz
0.4843397766496871


array([-0.15836366,  0.01133166,  0.29158731])